In [2]:
from pizzaproducer import PizzaProvider
from config import my_server
import random
from faker import Faker

In [3]:
fake=Faker()
#for the same result
Faker.seed(4321)

MAX_NUMBER_PIZZAS_IN_ORDER = 10
MAX_ADDITIONAL_TOPPINGS_IN_PIZZA = 5

In [4]:
import time
import json
from kafka import KafkaProducer

folderName = r"kafkaCerts/"

producer = KafkaProducer(
    #my_server has form <INSTANCE_NAME>-<PROJECT_NAME>.aivencloud.come:<PORT>, and it is Service URI 
    bootstrap_servers=my_server,
    security_protocol="SSL",
    ssl_cafile=folderName+"ca.pem",
    ssl_certfile=folderName+"service.cert",
    ssl_keyfile=folderName+"service.key",
    value_serializer=lambda v: json.dumps(v).encode('ascii'),
    key_serializer=lambda v: json.dumps(v).encode('ascii')
)

In [5]:
topic_name="pizza"

fake.add_provider(PizzaProvider)

for i in range(0,10):
    print(fake.pizza_name())

Salami
Pepperoni
Pepperoni
Marinara
Margherita
Marinara
Marinara
Salami
Salami
Margherita


In [6]:
def produce_pizza_order (FakerInstance,orderid = 1):
    shop = FakerInstance.pizza_shop()
    # Each Order can have 1-10 pizzas in it
    pizzas = []
    for pizza in range(random.randint(1, MAX_NUMBER_PIZZAS_IN_ORDER)):
        # Each Pizza can have 0-5 additional toppings on it
        toppings = []
        for topping in range(random.randint(0, MAX_ADDITIONAL_TOPPINGS_IN_PIZZA)):
            toppings.append(FakerInstance.pizza_topping())
        pizzas.append({
            'pizzaName': FakerInstance.pizza_name(),
            'additionalToppings': toppings
        })
    # message composition
    message = {
        'id': orderid,
        'shop': shop,
        'name': FakerInstance.unique.name(),
        'phoneNumber': FakerInstance.unique.phone_number(),
        'address': FakerInstance.address(),
        'pizzas': pizzas
    }
    key = {'shop': shop}
    return message, key

In [7]:
while i <  100:
    message, key = produce_pizza_order(fake,i)

    print("Sending: {}".format(message))
    # sending the message to Kafka
    producer.send(topic_name,
                  key=key,
                  value=message)
    # 2 seconds of sleep time before the next message
    time.sleep(2)

    # Force sending of all messages
    if (i % 100) == 0:
        producer.flush()
    i=i+1
producer.flush()

Sending: {'id': 9, 'shop': 'Marios Pizza', 'name': 'Jason Brown', 'phoneNumber': '(510)290-7469', 'address': '2701 Samuel Summit Suite 938\nRyanbury, MH 78954', 'pizzas': [{'pizzaName': 'Marinara', 'additionalToppings': ['🫑 green peppers', '🍍 pineapple', '🥚 egg']}]}
Sending: {'id': 10, 'shop': 'Ill Make You a Pizza You Cant Refuse', 'name': 'Edward Liu', 'phoneNumber': '(661)154-9335x314', 'address': '561 Lester Points Apt. 140\nEast Katelynland, NC 75965', 'pizzas': [{'pizzaName': 'Diavola', 'additionalToppings': ['🐟 tuna']}, {'pizzaName': 'Marinara', 'additionalToppings': ['🥚 egg', '🍅 tomato', '🥓 bacon']}, {'pizzaName': 'Salami', 'additionalToppings': ['🍓 strawberry', '🫑 green peppers', '🫒 olives', '🍍 pineapple']}, {'pizzaName': 'Diavola', 'additionalToppings': ['🍓 strawberry']}, {'pizzaName': 'Mari & Monti', 'additionalToppings': ['🐟 tuna', '🫑 green peppers', '🥓 bacon', '🧄 garlic']}, {'pizzaName': 'Margherita', 'additionalToppings': ['🍓 strawberry']}, {'pizzaName': 'Marinara', 'addi

KeyboardInterrupt: 